Kita h5pyd is still on 0.3.2, so need to pip install latest version of h5pyd:
```
pip install git+https://github.com/HDFGroup/h5pyd.git
```

In [1]:
# !pip install git+https://github.com/HDFGroup/h5pyd.git 

In [2]:
%matplotlib inline
import h5pyd as h5py
import numpy as np

In [3]:
print(h5py.__version__)

0.4.1


In [4]:
! hsls -H -v /home/jreadey/ESIP/

jreadey@hdfgroup.org                 folder   2019-05-04 14:15:37 /home/jreadey/ESIP/
jreadey@hdfgroup.org   104.1K        domain   2019-06-06 20:12:39 /home/jreadey/ESIP/adcirc_01.nc
jreadey@hdfgroup.org     3.6G        domain   2019-06-06 21:36:46 /home/jreadey/ESIP/adcirc_native.nc
3 items


## Try reading with h5pyd (0.4.0 version)

In [5]:
f = h5py.File("/home/jreadey/ESIP/adcirc_01.nc", "r")

In [6]:
zeta = f['zeta']

In [7]:
zeta.chunks

(10, 141973)

In [8]:
zeta.shape

(720, 9228245)

Let's try reading one chunk of data

In [9]:
n = 141973

In [10]:
%time arr = f['zeta'][:320, 2*n:3*n]

CPU times: user 896 ms, sys: 1.28 s, total: 2.18 s
Wall time: 11.7 s


How many chunks are there?

In [11]:
f['zeta']

<HDF5 dataset "zeta": shape (720, 9228245), type "<f8">

In [12]:
nchunks = np.prod(zeta.shape) / np.prod(zeta.chunks)
print(nchunks)

4680.0


In [13]:
f['zeta']

<HDF5 dataset "zeta": shape (720, 9228245), type "<f8">

In [14]:
np.prod(zeta.chunks)*8/1e6

11.35784

## Now try reading with xarray (using h5netcdf + h5pyd)

In [15]:
import xarray as xr

In [16]:
url_hsds = 'http://hsdshdflab.hdfgroup.org/home/jreadey/ESIP/adcirc_01.nc'
url_hsds = "http://hsdshdflab.hdfgroup.org/home/jreadey/ESIP/adcirc_native.nc"

In [17]:
from dask.distributed import Client, progress, LocalCluster
from dask_kubernetes import KubeCluster

In [18]:
cluster = LocalCluster()

In [19]:
cluster

In [20]:
client = Client(cluster)

In [21]:
ds_hsds = xr.open_dataset(url_hsds, engine='h5netcdf', chunks={'time':10, 'node':141973})

In [22]:
ds_hsds['zeta']

<xarray.DataArray 'zeta' (time: 720, node: 9228245)>
dask.array<shape=(720, 9228245), dtype=float64, chunksize=(10, 141973)>
Coordinates:
  * time     (time) datetime64[ns] 2031-08-03T02:10:00 ... 2031-08-08T02:00:00
    x        (node) float64 dask.array<shape=(9228245,), chunksize=(141973,)>
    y        (node) float64 dask.array<shape=(9228245,), chunksize=(141973,)>
Dimensions without coordinates: node
Attributes:
    long_name:      water surface elevation above geoid
    standard_name:  sea_surface_height_above_geoid
    location:       node
    mesh:           adcirc_mesh
    units:          m

In [23]:
%%time
max_var = ds_hsds['zeta'].max(dim='time').compute()

CPU times: user 2min 11s, sys: 11.9 s, total: 2min 23s
Wall time: 12min 46s
